In [1]:
import numpy as np
import hockey.hockey_env as h_env
import gymnasium as gym
from importlib import reload
import time
import torch


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
from torch.distributions import Categorical

import gymnasium as gym
import pygame
import torch

import math

import numpy as np
import collections, random

import os
import sys

import config

try:
    base_path = os.path.dirname(__file__)
except NameError:
    base_path = os.getcwd()

parent_dir = os.path.abspath(os.path.join(base_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from models.memory import ReplayBuffer
from models.policy_net import PolicyNet
from models.value_net import QNet, compute_target



Using device: cpu


In [2]:
np.set_printoptions(suppress=True)

reload(h_env)

/Users/gauravniranjan/anaconda3/envs/reinforcement_learning/lib/python3.9/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/gauravniranjan/anaconda3/envs/reinforcement_learning/lib/python3.9/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


<module 'hockey.hockey_env' from '/Users/gauravniranjan/anaconda3/envs/reinforcement_learning/lib/python3.9/site-packages/hockey/hockey_env.py'>

In [3]:
model_dir = 'runs/TE_4_2025-02-21_23-30-13/models/policy_net.pth'

In [4]:
#Since we changed some layer names, we need to rename the old model state dictionary to have the new layer names:
def load_new_dict(old_state_dict):

    new_state_dict = {}
    
    for key, value in old_state_dict.items():
        new_key = key
        if key.startswith("fc1"):
            new_key = key.replace("fc1", "feature_extractor")
        elif key.startswith("fc_mu"):
            new_key = key.replace("fc_mu", "mean_branch")
        elif key.startswith("fc_std"):
            new_key = key.replace("fc_std", "std_branch")
        new_state_dict[new_key] = value
        
    return new_state_dict

In [10]:
def play_hockey(opponent='weak'):
    env = h_env.HockeyEnv()
    weak_opponent = h_env.BasicOpponent(weak=True)
    strong_opponent = h_env.BasicOpponent(weak=False)
    state_dim = env.observation_space.shape[0]
    action_dim = env.num_actions
    
    chosen_opponent = weak_opponent
    
    if opponent == 'strong':
        chosen_opponent = strong_opponent

    print(f'state_dim: {state_dim},  action_dim: {action_dim}')


    pi1 = PolicyNet(config.lr_pi, state_dim, action_dim, config).to(config.device)
    
    #load old model state dict into new model (only the layer names are changed)
    pi1.load_state_dict(load_new_dict(torch.load(model_dir, map_location=config.device)))
    pi1.eval()



    for n_epi in range(5):
        s, _ = env.reset()
        done = False
        count = 0

        score = 0.0

        while count < 250 and not done:


            a, log_prob= pi1(torch.from_numpy(s).float())
            a = a.cpu().detach().numpy()


            a2 = chosen_opponent.act(env.obs_agent_two())

            s_prime, r, done, truncated, info = env.step(np.hstack([a,a2]))

            time.sleep(0.0)

            s = s_prime
            score += r


            env.render()

        print(f"Total score in the episode: {score}")
        
    env.close()


In [11]:
play_hockey(opponent='weak')

state_dim: 18,  action_dim: 4


/var/folders/2p/m351_1_j0s78h8kll_wpyh5w0000gn/T/ipykernel_20729/3169660563.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi1.load_state_dict(load_new_dict(torch.load

Total score in the episode: 9.933419529548235
Total score in the episode: 9.310111466444315
Total score in the episode: 9.909089329185335
Total score in the episode: 9.685592717464392
Total score in the episode: 9.867213957041539
